In [125]:
# All Imports
import pyttsx3 as tts
import speech_recognition as sr
import PyDictionary as pyd
from email.message import EmailMessage
from newsapi import NewsApiClient
import datetime
import wikipedia
import webbrowser
import pytube
import requests
import os
import json
import smtplib
import ssl
import randfacts
import wolframalpha
import re
import pyjokes
import dadjokes

# Web Crawling
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


In [126]:
# Global Variables
user_profile = json.load(open('data/user_profile.json'))["user"]
contacts = json.load(open('data/user_profile.json'))["contacts"]
settings = json.load(open("data/va_settings.json"))
stored_Websites = settings["websites"]
stored_API = settings["api"]


In [127]:
# Setting up the engine
engine = tts.init('sapi5')
voices = engine.getProperty('voices')
engine.setProperty('rate', 220)
engine.setProperty('voice', voices[1].id)


In [128]:
def speak(text):
    """Inputted text is spoken"""
    engine.say(text)
    print("Hazel: " + text)
    engine.runAndWait()


In [129]:
def wishMe():
    """Greet the User"""
    hour = int(datetime.datetime.now().hour)
    if hour >= 0 and hour < 12:
        speak("Good Morning! ")
    elif hour >= 12 and hour < 18:
        speak("Good Afternoon! ")
    else:
        speak("Good Evening!")

    speak("I am Hazel. Please tell me how may I help you")


In [130]:
def acceptCommand():
    """Accepts the user's command via the microphone and returns the string output"""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.adjust_for_ambient_noise(source, duration=0.5)
        r.pause_threshold = 1
        # seconds of non-speaking audio before a phrase is considered complete
        audio = r.listen(source)

    try:
        query = r.recognize_google(audio, language=settings["language"])
        print(user_profile["user"]["name"]+": ", query, end="\n\n")
    except Exception as e:
        speak("Unable to recognize your voice. Please say that again...")
        return "None"
    return query


In [131]:
def sendMail():
    sender = user_profile["email"]
    password = user_profile["password"]
    receiver = contacts["Uzair"]["email"]

    em = EmailMessage()
    em['From'] = sender
    em['To'] = receiver
    em['Subject'] = "Testing"
    em.set_content("I am Tired.")

    context = ssl.create_default_context()
    with smtplib.SMTP_SSL('smtp.gmail.com', 465, context=context) as smtp:
        smtp.login(sender, password)
        smtp.sendmail(sender, receiver, em.as_string())


In [132]:
def searchWikipedia():
    """Searches Wikipedia for the user's query"""
    query = acceptCommand().lower()
    q = wikipedia.search(query)[0]
    print("Searched", q)
    results = wikipedia.summary(q, sentences=1, auto_suggest=False)
    # results = wikipedia.summary(query, sentences=2)
    speak("According to Wikipedia")
    print(results)
    speak(results)


In [140]:
def getFacts():
    """Returns a random fact"""
    speak("Here is a random fact")
    speak(randfacts.get_fact())


def getJokes():
    """Returns a random joke"""
    speak("Here is a random joke")
    joking = dadjokes.Dadjoke()
    speak(joking.joke)


In [141]:
def playMusic():
    """Plays music from the user's directory"""
    #! Still has some issues with song title match
    songs = [s.lower() for s in os.listdir(
        "D:/Transfer/Music/LS-F-KPS-320/LS-F-KPS-320")]
    print(songs)
    speak("Which song would you like to play?")
    query = acceptCommand().lower()
    if query+".mp3" in songs:
        os.startfile(
            "D:/Transfer/Music/LS-F-KPS-320/LS-F-KPS-320/" + query + ".mp3")
    else:
        speak("I don't know that song")


In [142]:
def playYouTube():#TODO Incomplete
    """Plays a YouTube video from the user's query"""
    query = acceptCommand().lower()
    query = re.sub(r"\W+", "", query)
    yt = pytube.YouTube("https://www.youtube.com/watch?v=" + query)
    # stream = yt.streams.first()
    # stream.download("D:/Transfer/Music/YouTube")
    # os.startfile("D:/Transfer/Music/YouTube/" + query + ".mp4")


In [136]:
def solveProblems():
    """Perform basic problems solving"""
    client = wolframalpha.Client(stored_API["wolframalpha"]["client"])
    query = acceptCommand().lower()
    response = client.query(query)
    try:
        answer = next(response.results).text
        speak(answer)
    except Exception as e:
        speak("I don't know that")


In [137]:
def useDictionary():
    """Get Meaning, Synonyms, Antonyms"""
    query = acceptCommand().lower()
    dictionary = pyd.PyDictionary()
    try:
        # TODO Needs more polishing on the response part and command
        meaning = dictionary.meaning(query)
        speak("According to dictionary")
        print(meaning)
        speak(meaning)
    except Exception as e:
        speak("I don't know that")


In [138]:
def openWebsite():
    """Opens a website from the user's query"""
    query = acceptCommand().lower()
    if stored_Websites.__contains__(query):
        webbrowser.open(stored_Websites[query])
    else:
        speak(
            "I don't know that website. You can add the URL to the list of websites I know")
        if input("Want to add Website to list? (y/n) ") == "y":
            stored_Websites[input("Enter Website Name: ")
                            ] = input("\nEnter URL: ")
            speak("Website added to list")
            openWebsite()
        # speak("What is the URL of the website you want to open?")
        # input()


In [143]:
#TODO Incomplete

# def getNews():
client = NewsApiClient(api_key=stored_API['news']['client'])
# client.get_top_headlines(country="in", language=settings["language"])
